In [1]:
import torch
import numpy as np
from datasets import load_dataset
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from warnings import filterwarnings
filterwarnings('ignore')

import torch._dynamo
torch._dynamo.config.suppress_errors = True

/home/arjun/Desktop/AI_ENV/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 256
n = 5
embedding_dims = 10 # how many dimensional vector should represent each word in the vocabulory
num_epochs = 1000
lr = 0.01
print(device)

cuda


# Creating vocabulory

In [3]:
ds = load_dataset("MuskumPillerum/General-Knowledge")
df = ds['train'][:10]

In [4]:
def tokenize(sentence):
    data = sentence
    split_terms = [',', '.', '!', '?', '(', ')', '&', '$', '+', '-', '/', '*', ';', ':']
    for split_term in split_terms:
        if split_term in sentence:
            data = data.replace(split_term, f' {split_term} ')
    data = data.split()
    return data

tokenize('what is your !! ! name ?    ')

['what', 'is', 'your', '!', '!', '!', 'name', '?']

In [5]:
vocab_list = set()
X = []
for x,y in zip(df['Question'], df['Answer']):
    data = f'<s> Question: {x} Answer: {y} </s>'
    data = data.lower().replace('\\n', '')
    vocab_list.update(tokenize(data))
    X.append(data)

In [6]:
vocab_list.add('<UNK>')

In [7]:
vocab = {v:i for v,i in zip(vocab_list, range(0, len(vocab_list)+1))}
len(vocab)

145

In [8]:
embedding_matrix = torch.randn(len(vocab),embedding_dims)
embedding_matrix.shape

torch.Size([145, 10])

In [9]:
def get_word_embedding(word, vocab=vocab, embedding_matrix=embedding_matrix):
    if word not in vocab:
        word = '<UNK>'
    embedding = embedding_matrix[vocab[word]]
    return embedding

get_word_embedding('as')

tensor([-0.6791,  0.5203, -0.0739,  2.5437, -0.3951, -0.7738,  0.3530, -0.8437,
         0.2350, -0.6342])

# Creating the model

In [10]:
class NeuralNetwork(nn.Module):
    def __init__(self, n=n, num_hidden_layer=1024, vocab = vocab, dim_embedding=10):
        super(NeuralNetwork, self).__init__()
        self.rev_vocab = {v:k for k,v in vocab.items()}
        vocab_len = len(vocab)
        self.n = n
        self.dim_embedding = dim_embedding
        self.embedding = nn.Embedding(vocab_len, dim_embedding).to(device)

        self.hidden_layer = nn.Linear((n-1)*dim_embedding, num_hidden_layer)
        self.relu = nn.ReLU()
        self.output = nn.Linear(num_hidden_layer, vocab_len)

    def forward(self,x):
        # x will be the indices of embedding representing the input words
        x = torch.tensor(x)
        # print(x.shape)
        x_embeddings = self.embedding(x).view(-1,(self.n-1)*self.dim_embedding).to(device)
        out = self.hidden_layer(x_embeddings)
        out = self.relu(out)
        out = self.output(out)
        return out
    

    def generate(self, x):
        x = torch.tensor(x).to(device)

        # print(x.shape)
        # assert x.shape == 0 # fix shape here, and write a loop to do autoregressive text gen
        x = x[-4:]
        assert len(x) == self.n - 1
        logits = self.forward(x)
        probs = torch.softmax(logits, dim=1)
        prob, predicted_token_index = torch.max(probs, dim=1)
        return predicted_token_index

model = NeuralNetwork(n=n).to(device)

# Creating dataset

In [11]:
dataset = []
for x,y in zip(df['Question'], df['Answer']):
    data = f'Question: {x} Answer: {y}'.lower()
    tokenized_data = tokenize(data)
    for i in range(len(tokenized_data)-n):
        # print(i)
        data_i = tokenized_data[i:i+n]
        dataset.append([vocab[i] if i in vocab else vocab['<UNK>'] for i in data_i])
    # print()

dataset_np = np.array(dataset)
dataset_np.shape

(399, 5)

In [12]:
class QuestionAnswerDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.x = dataset[:,[i for i in range(n-1)]]
        self.y = dataset[:,-1]
        self.m, self.n = self.x.shape
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.m
    
dataset = QuestionAnswerDataset(dataset=dataset_np)
dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0)

# Training

In [13]:
model = model.to(device)
model = torch.compile(model)
lossCategory = nn.CrossEntropyLoss()
optimiser = torch.optim.SGD(model.parameters(), lr=lr)


In [14]:
torch.set_float32_matmul_precision('high')

In [15]:
for epoch in range(num_epochs):
    for i, (input, label) in enumerate(dataloader):
        x = input.to(device)
        y = label.to(device)
        y_pred = model(x)
        loss = lossCategory(y_pred, y)
        if epoch%100==0 and i==0: 
            print(f'Epoch {epoch} Loss: {loss.item()}')
        loss.backward()
        optimiser.step()
        optimiser.zero_grad()


Epoch 0 Loss: 4.9908952713012695
Epoch 100 Loss: 3.531843900680542
Epoch 200 Loss: 2.988238573074341
Epoch 300 Loss: 2.392522096633911
Epoch 400 Loss: 1.9637738466262817
Epoch 500 Loss: 1.5957809686660767
Epoch 600 Loss: 1.3935860395431519
Epoch 700 Loss: 1.1414200067520142
Epoch 800 Loss: 0.9531048536300659
Epoch 900 Loss: 0.7967060804367065


# Inference

In [16]:
text = '<s> Question: What is deep learning?'
tokenized_text = tokenize(text.lower())
embedding_indices = torch.tensor(np.array([vocab[word] for word in tokenized_text])).to(device)
embedding_indices

tensor([  0,  63, 135,  86,  46,  68,  62, 118], device='cuda:0')

In [17]:
vocab_rev = {v:k for k,v in vocab.items()}

In [18]:
gen_word = ''
input_embeddings = embedding_indices
max_gen = 30
words_generated = 0
print(text, end = ' ')

while gen_word != '</s>' and words_generated <= max_gen:
    gen_word = model.generate(input_embeddings)
    words_generated += 1
    input_embeddings = list(input_embeddings)
    input_embeddings.append(gen_word.item())
    print(vocab_rev[gen_word.item()], end= ' ')



<s> Question: What is deep learning? answer : deep learning is a subset of artificial intelligence that focuses on enabling computers to understand and and visual data . the world around us . the data . the 

In [19]:
# TO DO
# normalisation, standardise, proper init, weight decay, hyperparameter tunign, diff optimiser test, inference, earlyy stopping
# add direct connections, see if it is useful
# plot unigram, bi, tri, etc perplexity score, word error rate, etc